In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import functions as func

Custom Funtions

In [2]:

model=YOLO('models/yolo11l-pose.pt')

predict=model(source='images/image_7.jpg')


image 1/1 /Users/manohar/Documents/GCP_ML_Engineer/Code/Lab/9_Image_Ergonomics/images/image_7.jpg: 384x640 2 persons, 145.7ms
Speed: 1.0ms preprocess, 145.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)


In [3]:
predict[0].keypoints

ultralytics.engine.results.Keypoints object with attributes:

conf: tensor([[0.9905, 0.5967, 0.9964, 0.0183, 0.9907, 0.8593, 0.9829, 0.2782, 0.9577, 0.3855, 0.9608, 0.0701, 0.1545, 0.0062, 0.0169, 0.0017, 0.0026],
        [0.9907, 0.9126, 0.9866, 0.3312, 0.9780, 0.9972, 0.9876, 0.9719, 0.8818, 0.8879, 0.7612, 0.9474, 0.9067, 0.1120, 0.0654, 0.0044, 0.0027]])
data: tensor([[[5.1684e+02, 4.1999e+02, 9.9049e-01],
         [5.1512e+02, 3.6402e+02, 5.9674e-01],
         [4.6156e+02, 3.7569e+02, 9.9645e-01],
         [4.4690e+02, 3.6348e+02, 1.8322e-02],
         [2.9939e+02, 3.9711e+02, 9.9070e-01],
         [3.8388e+02, 6.2787e+02, 8.5929e-01],
         [2.5503e+02, 6.6315e+02, 9.8289e-01],
         [6.7762e+02, 9.2908e+02, 2.7825e-01],
         [7.0771e+02, 9.3964e+02, 9.5767e-01],
         [1.1399e+03, 9.6028e+02, 3.8553e-01],
         [1.1960e+03, 8.9946e+02, 9.6076e-01],
         [3.5704e+02, 1.1174e+03, 7.0073e-02],
         [2.4896e+02, 1.1250e+03, 1.5449e-01],
         [6.8758e+02, 

In [4]:
annotated_frame=predict[0].plot()


In [5]:

# For each detected person:

if predict[0].keypoints is not None:
    for kpts in predict[0].keypoints.data:
        # Extract points (in format x,y,score)
        kpts = kpts.cpu().numpy()

        LEFT_SHOULDER = kpts[5][:2]
        LEFT_ELBOW = kpts[7][:2]
        LEFT_WRIST = kpts[9][:2]

        if np.all(LEFT_SHOULDER) and np.all(LEFT_ELBOW) and np.all(LEFT_WRIST):
            angle = func.calculate_angle(LEFT_SHOULDER, LEFT_ELBOW, LEFT_WRIST)
            rula_score = func.get_rula_score(angle)
            risk = func.get_risk_label(rula_score)

            # Show on frame
            cv2.putText(annotated_frame, f"Angle: {int(angle)}°", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"RULA: {rula_score}", (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
            cv2.putText(annotated_frame, f"Risk: {risk}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)



In [ ]:

cv2.imshow("Ergonomic Posture Detection (YOLO-Pose)", annotated_frame)
# cv2.imwrite(cv2.imread(annotated_frame))

cv2.waitKey(0)
cv2.destroyAllWindows()

: 